## API Tiingo - Diario
###### Helga Zambrana - Marzo 2024

#### Importar las librerías necesarias

In [1]:
import requests
from decouple import config
import pandas as pd
import psycopg2
import datetime

#### Leer el usuario, la contraseña y el token desde la variable de entorno

In [2]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Obtener datos de Tiingo y mostrarlos en un dataframe

In [3]:
class ListaSimbolos:
    def __init__(self, simbolos):
        self.simbolos = simbolos
    
    def __call__(self):
        return self.simbolos

class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def __call__(self, lista_simbolos):
        simbolos = lista_simbolos()
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices"
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            
            # Filtrar las columnas no deseadas
            columnas_no_deseadas = ['adjClose', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor']
            df = df.drop(columns=columnas_no_deseadas, errors='ignore')
            
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'Fecha',
                'ticker': 'Símbolo',
                'open': 'Apertura',
                'high': 'Máximo',
                'low': 'Mínimo',
                'close': 'Cierre',
                'volume': 'Volumen',
            }
            df.rename(columns=columnas, inplace=True)
            
            # Convertir la columna de fecha a tipo datetime
            df['Fecha'] = pd.to_datetime(df['Fecha'])
            
            # Llamar a la instancia de ListaSimbolos para imprimir los símbolos
            print(lista_simbolos())
            
            return df
        else:
            return None


# Lista de símbolos a consultar
simbolos = ListaSimbolos(['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA'])


# Crear instancia de TiingoData
tiingo_data = TiingoData(token)


# Obtener y mostrar los datos
datos_tiingo_df = tiingo_data(simbolos)
if datos_tiingo_df is not None:
    print(datos_tiingo_df)

['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA']
                         Fecha  Cierre  Máximo   Mínimo  Apertura    Volumen  \
0    2023-01-03 00:00:00+00:00   85.82   86.96   84.205     85.46   76706040   
1    2023-01-04 00:00:00+00:00   85.14   86.98   83.360     86.55   68885123   
2    2023-01-05 00:00:00+00:00   83.12   85.42   83.070     85.33   67930825   
3    2023-01-06 00:00:00+00:00   86.08   86.40   81.430     83.03   83303361   
4    2023-01-09 00:00:00+00:00   87.36   89.48   87.080     87.46   65266056   
...                        ...     ...     ...      ...       ...        ...   
3155 2024-04-01 00:00:00+00:00  175.22  176.75  170.210    176.17   80751684   
3156 2024-04-02 00:00:00+00:00  166.63  167.69  163.430    164.75  115344925   
3157 2024-04-03 00:00:00+00:00  168.38  168.82  163.280    164.02   82223543   
3158 2024-04-04 00:00:00+00:00  171.11  177.19  168.010    170.07  122061224   
3159 2024-04-05 00:00:00+00:00  164.90  1

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [4]:
class BaseDatosTiingo:
    def __init__(self, host, port, database_name, user, password):
        self.host = host
        self.port = port
        self.database_name = database_name
        self.user = user
        self.password = password
        self.conn = None

    def conectar(self):
        try:
            self.conn = psycopg2.connect(
                host=self.host, 
                port=self.port, 
                database=self.database_name, 
                user=self.user, 
                password=self.password
            )
            print("Conexión a la base de datos exitosa.")
        except Exception as e:
            print(f"Error al crear la conexión a la base de datos:", e)

    def crear_tabla_tiingo(self):
        if self.conn is not None:
            try:
                cur = self.conn.cursor()
                cur.execute("""
                CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.tiingo (
                    simbolo VARCHAR(10) DISTKEY NOT NULL,
                    fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
                    fecha TIMESTAMP SORTKEY NOT NULL,
                    apertura FLOAT(15) NULL,
                    maximo FLOAT(15) NULL,
                    minimo FLOAT(15) NULL,
                    cierre FLOAT(15) NULL,
                    volumen INTEGER NULL
                );
                """)
                self.conn.commit()
                print("Tabla creada exitosamente.")
            except Exception as e:
                print(f"Error al crear la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

    def cerrar_conexion(self):
        if self.conn is not None:
            self.conn.close()
            print("Conexión cerrada correctamente.")
        else:
            print("No hay conexión para cerrar.")

# Crear instancia de la clase BaseDatosTiingo con los datos de conexión de .env
bd_tiingo = BaseDatosTiingo(my_host, port, database_name, user, password)

# Conectar con la base de datos
bd_tiingo.conectar()

# Crear tabla en la base de datos
bd_tiingo.crear_tabla_tiingo()

# Cerrar conexión
bd_tiingo.cerrar_conexion()

Conexión a la base de datos exitosa.
Tabla creada exitosamente.
Conexión cerrada correctamente.
